## HW 1 ##

In [0]:
import numpy as np
from scipy import linalg
import random
from itertools import combinations 

### Problem 2 ###

**Some definitions**

Let 

$$M\in\mathbb{R}^{n\times n}$$ be an arbitrary matrix.  

Let $$p(x)=a_0 + a_1 x + a_2 x^2 + \ldots + a_n x^n \in\mathbb{R}[x]$$ be an arbitrary polynomial of less or equal to $n$.

The above polynomial can be used to define a matrix function that takes matrices as input and outputs matrices as follows: 

$$p(M) = a_0 I + a_1 M + \ldots + a_n M^n,$$ 

that is, each monomial $x^k$ is substituted by the corresponding matrix power $M^k$.

We say that a polynomial $p(x)$ annihilates a matrix $M\in\mathbb{R}^{n\times n}$ iff $p(M)=\boldsymbol{0}$, where $\boldsymbol{0}$ is the zero matrix.

**Task**

The task is to write a function ```annihilate_poly``` that takes as input an arbitrary square numpy array $M$ and outputs a vector whose cofficients are the coefficients of a (non-trivial) polynomial that annihilates $M$.  One-trivial means that its is not the zero polynomial which maps every matrix to the zero matrix.

**Hint**

You can reduce the problem to finding a linear dependance relationship between the $n+1$ vectors 

$$\mathrm{vec}(I), \mathrm{vec}(M), \mathrm{vec}(M^2),\ldots,\mathrm{vec}(M^n)\in\mathrm{R}^{n^2}.$$



The operation $\mathrm{vec}$ turns a square matrix $M\in\mathbb{R}^{n\times n}$ into a vector $v\in\mathbb{R}^{n^2}$ by first listing the entries of the first row, then those of the second row etc.

Update: 

To solve this problem, you have to compute the null space of the matrix $A\in \mathbb{R}^{n^2\times (n+1)}$ whose columns are the vectors $\mathrm{vec}(M^k)$ for $k\in\{0,\ldots,n\}$.


(This is not needed: 

If you don't remeber how to compute the find a linear dependance relationship, check out this stackoverflow post: https://math.stackexchange.com/questions/2198960/finding-linear-dependence-relation

You can use https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.solve.html to solve the resulting matrix equation.)

In [0]:
def annihilate_poly(M):
  '''
  Return a basis for all the vectors whose coefficients form a non-trivial polynomial that annihilate the matrix M
  '''

  M = np.matrix(M)

  # Check that M is a square matrix
  if M.shape[0] != M.shape[1]:
    raise ValueError('Not a square matrix')

  n = M.shape[0]

  # Build matrix A, whose columns are the vectors vec(M^k) for 0 <= k <= n
  A = np.empty((n**2, n+1), dtype=int)
  for i in range(n+1):
    col = (M**i).reshape((n**2,))
    A[:, i] = col

  # This will return a basis for the null space of A
  # From this, we could constuct an infinite number of annihilating polynomials
  ps = linalg.null_space(A)

  # However, we only need a single annihilating polynomial
  # Since any of the vectors from this basis will form annihilating polynomials, I can simply return any of the vectors 
  
  # Randomly choose a vector from the basis and return it
  rand = random.randint(0, ps.shape[1]-1)
  return ps[:, rand]

Below I will demonstrate that my function can successfully find annihilating polynomials

In [14]:
M = np.matrix([[1,0], [0,1]])
print('M: \n{}\n'.format(M))

p = annihilate_poly(M)
print('Annihilating polynomial:\n{}'.format(np.poly1d(p[::-1])))

M: 
[[1 0]
 [0 1]]

Annihilating polynomial:
        2
0.7071 x - 0.7071 x


To prove my function is outputting annihilating polynomials, I create a function which plugs in matrix M into the polynomial and checks whether the result is equal to the zero matrix.

In [0]:
def check_annihilates(M, p, verbose=True):
  '''
  Checks whether a polynomial p annihilates the matrix M
  '''

  result = 0

  # Creating a string to represent the polynomial

  # Plug matrix M into p
  for i, coef in enumerate(p):
    result += coef * (M**i)

  # Check whether the result is equal to the zero matrix
  if np.array_equal(np.round(result, 5), np.zeros_like(result)):
    if verbose:
      print('Success!\n {} is an annihilating polynomial \n'.format(np.poly1d(p[::-1])))
    else:
      return True
  else:
    if verbose:
      print('Failure!\n {} is not an annihilating polynomial \n'.format(np.poly1d(p[::-1])))
    else:
      return False

In [16]:
check_annihilates(M, p, verbose=True)

Success!
         2
0.7071 x - 0.7071 x is an annihilating polynomial 



To ensure my function works properly, I will now generate a random matrix and use my function to generate an annihilating polynomial, then prove it is correct.

In [17]:
# Generate a random square matrix, anywhere from shape (1,1) to (7,7)
n = random.randint(1,7)
M = np.matrix([np.random.randint(10, size = n) for _ in range(n)])
print('M:\n{}\n'.format(M))

# Find annihilating polynomial
p = annihilate_poly(M)
print('Annihilating polynomial:\n{}\n'.format(np.poly1d(p[::-1])))

check_annihilates(M, p, verbose=True)

M:
[[9 8 7 9 4 4 6]
 [2 9 8 3 3 2 2]
 [4 1 2 6 3 3 3]
 [6 8 0 4 6 5 9]
 [5 7 6 4 5 3 5]
 [3 2 0 5 6 0 7]
 [2 7 6 8 0 5 5]]

Annihilating polynomial:
           7            6             5            4            3
1.127e-05 x - 0.000383 x + 0.0004393 x + 0.001701 x + 0.009339 x
            2
 - 0.03306 x - 0.2776 x + 0.9601

Success!
            7            6             5            4            3
1.127e-05 x - 0.000383 x + 0.0004393 x + 0.001701 x + 0.009339 x
            2
 - 0.03306 x - 0.2776 x + 0.9601 is an annihilating polynomial 



**Task**

Write a function ```annihilate_min_deg_poly``` that computes a non-trivial polynomial that annihilates a given square matrix and has the smallest possible degree.  Recall that a polynomial $p(x)$ has degree $d$ if the coefficient $a_{d+1}=\ldots=a_n=0$.

To find the annihilating polynomial with the smallest degree, rather than finding the null space of the matrix with columns vec(M^k), I will now begin by computing the eigenvalues of the input matrix M. From this, we can find the matrix's minimum annihilating polynomial. 

In order to find the minimum polynomial, I test all of the polynomials which divide the characteristic function, beginning with that of the lowest order, and stop once we encounter an annhilating polynomial.

In [0]:
def annihilate_min_deg_poly(M):
  M = np.matrix(M)

  # Check that M is a square matrix
  if M.shape[0] != M.shape[1]:
    raise ValueError('Not a square matrix')

  n = M.shape[0]

  # Get eigenvalues
  eigs = np.linalg.eigvals(M)
  
  # Create 1-d polynomials from eigenvalues
  polys = []
  for eig in eigs:
    polys.append(np.poly1d([1, -eig]))

  # Loop through each possible degree polynomial
  for i in range(1, M.shape[0]+1):

    # Create all possible polynomials of degree i which would divide the characteristic equation
    combs = combinations(polys, i) 

    # Check each polynomial to see if it's an annhihilating poly
    for comb in list(combs):
      # This is simply multiplying (FOIL) out the 1-d polynomials 
      lowest_degree_p = np.poly1d([1])
      for ele in comb:
        lowest_degree_p *= ele
      
      # Need to convert back to list representation for my check_annihilates() function to work
      lowest_degree_p = list(lowest_degree_p)[::-1]
      if check_annihilates(M, lowest_degree_p, verbose=False):
        return lowest_degree_p
      
  return None

The test below with show that this approach still gets us an annihilating polynomial. Following this test, I will run three additional tests which will show that this function is in fact generating **minimum** annihilating polynomials.

In [28]:
# Generate a random square matrix, anywhere from shape (1,1) to (5,5)
n = random.randint(1,5)
M = np.matrix([np.random.randint(10, size = n) for _ in range(n)])
print('M:\n{}\n'.format(M))

# Find minimum annihilating polynomial
lowest_degree_p = annihilate_min_deg_poly(M)
print('Annihilating polynomial:\n{}\n'.format(np.poly1d(lowest_degree_p[::-1])))

check_annihilates(M, lowest_degree_p, verbose=True)

M:
[[1 5 7 2 8]
 [0 8 6 0 2]
 [0 5 6 9 4]
 [1 8 5 6 3]
 [9 5 2 8 5]]

Annihilating polynomial:
   5      4      3       2
1 x - 26 x + 66 x + 461 x - 1103 x + 6084

Success!
    5      4      3       2
1 x - 26 x + 66 x + 461 x - 1103 x + 6084 is an annihilating polynomial 



I'll finally run three more tests to show my function generates **minimum** annhihilating polynomials. These three test matrices are diag(a, a, a), diag(a, a, c), and diag(a, b, c), where a, b, and c are three distinct numbers not equal to zero. 

I'll then ensure my function correctly outputs the following three minimal polyomials: x - a, (x - a) * (x - c), and (x - a) * (x - b) * (x - c).

In [21]:
a, b, c = random.sample(range(1, 20), 3)

print('a: {}, b: {}, c: {}'.format(a, b, c))

a: 6, b: 13, c: 5


In [22]:
# Test 1
M = np.matrix([[a, 0, 0], [0, a, 0], [0, 0, a]])
print('M:\n{}\n'.format(M))

# Find minimum annihilating polynomial
lowest_degree_p = annihilate_min_deg_poly(M)
print('Annihilating polynomial:{}\n'.format(np.poly1d(lowest_degree_p[::-1])))

# Ensure we've got an annhiliating polynomial
check_annihilates(M, lowest_degree_p)

if np.array_equal(lowest_degree_p, np.array([-a, 1])):
  print('Test successful! The minimum annhilating polynomial is x - a, or {}'.format(np.poly1d(lowest_degree_p[::-1])))
else:
  print('Test unsuccessful.\n {} was not the minimum annihilating polynomial'.format(np.poly1d(lowest_degree_p[::-1])))

M:
[[6 0 0]
 [0 6 0]
 [0 0 6]]

Annihilating polynomial: 
1 x - 6

Success!
  
1 x - 6 is an annihilating polynomial 

Test successful! The minimum annhilating polynomial is x - a, or  
1 x - 6


In [23]:
# Test 2
M = np.matrix([[a, 0, 0], [0, a, 0], [0, 0, c]])
print('M:\n{}\n'.format(M))

# Find minimum annihilating polynomial
lowest_degree_p = annihilate_min_deg_poly(M)
print('Annihilating polynomial:\n{}\n'.format(np.poly1d(lowest_degree_p[::-1])))

if np.array_equal(lowest_degree_p, np.array([a*c, -a-c, 1])):
  print('Test successful! The minimum annhilating polynomial is (x - a) * (x - c), or \n {}'.format(np.poly1d(lowest_degree_p[::-1])))
else:
  print('Test unsuccessful. {} was not the minimum annihilating polynomial'.format(np.poly1d(lowest_degree_p[::-1])))

M:
[[6 0 0]
 [0 6 0]
 [0 0 5]]

Annihilating polynomial:
   2
1 x - 11 x + 30

Test successful! The minimum annhilating polynomial is (x - a) * (x - c), or 
    2
1 x - 11 x + 30


In [24]:
# Test 3
M = np.matrix([[a, 0, 0], [0, b, 0], [0, 0, c]])
print('M: \n{}\n'.format(M))

# Find minimum annihilating polynomial
lowest_degree_p = annihilate_min_deg_poly(M)
print('Annihilating polynomial coefficients: \n{}\n'.format(np.poly1d(lowest_degree_p[::-1])))

# Ensure its the minimum annihilating polynomial
if np.array_equal(lowest_degree_p, np.array([-a*b*c, a*b+a*c+b*c, -a-b-c, 1])):
  print('Test successful! The minimum annhilating polynomial is (x - a) * (x - b) * (x - c) or \n {}'.format(np.poly1d(lowest_degree_p[::-1])))
else:
  print('Test unsuccessful. {} was not the minimum annihilating polynomial'.format(np.poly1d(lowest_degree_p[::-1])))

M: 
[[ 6  0  0]
 [ 0 13  0]
 [ 0  0  5]]

Annihilating polynomial coefficients: 
   3      2
1 x - 24 x + 173 x - 390

Test successful! The minimum annhilating polynomial is (x - a) * (x - b) * (x - c) or 
    3      2
1 x - 24 x + 173 x - 390
